In [1]:
!pip install gym

    100% |████████████████████████████████| 1.5MB 12.2MB/s 
    100% |████████████████████████████████| 1.0MB 18.1MB/s 
  Running setup.py bdist_wheel for gym ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/6c/3a/0e/b86dee98876bb56cdb482cc1f72201035e46d1baf69d10d028
Successfully built gym


In [0]:
import tensorflow as tf
import numpy as np
import gym

In [11]:
# Create the env
env = gym.make('FrozenLake8x8-v0')

/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [0]:
# Get the action_space and observation_space
action_size = env.action_space.n
observation_size = env.observation_space.n

In [0]:
tf.reset_default_graph()

# The Q-Network

In [0]:
# Hyperparameters of the network
lr = 0.1
max_steps = 99
gamma = .99
total_episodes = 10000

# Exploration parameters
epsilon = 1.0
max_epsilon = 1.0
min_epsilon = 0.3
decay_rate = 0.01

In [0]:
# Declare the placeholders
inputs = tf.placeholder(shape=[1, observation_size], dtype=tf.float32)
tqvals = tf.placeholder(shape=[1, action_size], dtype=tf.float32)

In [57]:
# Build the model

W1 = tf.get_variable('qweights', shape=[observation_size, action_size], 
                     initializer=tf.random_normal_initializer())

pout = tf.matmul(inputs, W1)
predict = tf.argmax(pout, axis=1)

ValueError: ignored

In [0]:
# Define the loss function and optimizer
criterion = tf.reduce_sum(tf.square(out_action - pout))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
train_op = optimizer.minimize(criterion)

In [0]:
# Traning the network
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for episode in range(total_episodes):
        
        state = env.reset()
        done = False
        total_rewards = 0
        
        for step in range(max_steps):
            
            action, curr_qvals = sess.run([predict, pout], 
                                       feed_dict={inputs: np.identity(observation_size)[state:state+1]})
            
            exp_prob = np.random.uniform()
            if exp_prob < epsilon:
                action = env.action_space.sample()
                
            print (action, curr_qvals)
            
            # Step with the chosen action
            new_state, rewards, done, info = env.step(action)
            
            # Get the qvalues for the new state
            next_qvals = sess.run([pout], 
                         feed_dict={inputs: np.identity(observation_size)[new_state:new_state+1]})
            
            # Get the target output for current state
            max_next_qvals = np.max(next_qvals)
            target_q = curr_qvals
            target_q[0, action] = rewards + (gamma * max_next_qvals)
            
            # Train the network using the target Q values
            _ = sess.run([train_op], feed_dict={inputs: np.identity(observation_size)[state:state+1],
                                                tqvals: target_q})
            
            
            total_rewards += rewards
            
            if done or step == max_steps:
                env.render()
                print ('Maximum steps: {}'.format(step))
                break
                
            state = new_state
            
        reward_list.append(total_rewards)
        steps_list.append(step)
        
print ('Agent Trained successfully!!')
print ('Rewards over time: {:.3f}'.format(sum(reward_list) / total_episodes))

In [59]:
observation_space

64